In [2]:
# These magics make working in the notebook a little easier
%matplotlib inline
%load_ext autoreload
%autoreload 2

import torch
import paderbox as pb
from espnet2.tasks.asr import ASRTask
from espnet2.bin.asr_inference import Speech2Text

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
asr_train_config= '/net/vol/vivekkan/experiments/fearless/asr1/exp/asr_SpkVec/config.yaml'
asr_model_file= '/net/vol/vivekkan/experiments/fearless/asr1/exp/asr_SpkVec/valid.acc.ave.pth'
asr_model, asr_train_args = ASRTask.build_model_from_file(
            asr_train_config, asr_model_file, 'cpu')

In [4]:
from espnet2.tasks.lm import LMTask
lm_train_config= '/net/vol/vivekkan/experiments/wsj/asr1/exp/lm_train_lm_transformer_char/config.yaml'
lm_file = '/net/vol/vivekkan/experiments/wsj/asr1/exp/lm_train_lm_transformer_char/valid.loss.ave.pth'

lm, lm_train_args = LMTask.build_model_from_file(
                lm_train_config, lm_file, 'cpu'
            )

In [5]:
speech2text = Speech2Text(
        asr_train_config=asr_train_config,
        asr_model_file=asr_model_file,
        lm_train_config=lm_train_config,
        lm_file=lm_file,
        token_type='char',
        bpemodel=None,
        device='cpu',
        maxlenratio=0.0,
        minlenratio=0.0,
        dtype='float32',
        beam_size=20,
        ctc_weight=0.3,
        lm_weight=0.3,
        penalty=0.0,
        nbest=1,
    )

In [6]:
data_path_and_name_and_type = [['/net/vol/vivekkan/experiments/fearless/asr1/dump/raw/devset/wav.scp', 'speech', 'sound'], ['/net/vol/vivekkan/experiments/fearless/asr1/dump/raw/devset/utt2spkvec.scp', 'speaker_id', 'npy']]
import os
os.chdir('/net/home/vivekkan/my_project/espnet/egs2/fearless/asr1')
loader = ASRTask.build_streaming_iterator(
        data_path_and_name_and_type,
        dtype='float32',
        batch_size=5,
        key_file='/net/vol/vivekkan/experiments/fearless/asr1/exp/asr_SpkVec/decode_lm_lm_train_lm_transformer_char_valid.loss.ave_asr_model_valid.acc.ave/devset/logdir/keys.1.scp',
        num_workers=0,
        preprocess_fn=ASRTask.build_preprocess_fn(speech2text.asr_train_args, False),
        collate_fn=ASRTask.build_collate_fn(speech2text.asr_train_args, False),
        allow_variable_data_keys=True,
        inference=True)

In [7]:
keys, batch = next(iter(loader))

In [ ]:
batch['speech'].shape

In [8]:
batch.pop('speech_lengths')
batch.pop('speaker_id_lengths')

tensor([163, 163, 163, 163, 163])

In [10]:
asr_model

ESPnetASRModel(
  (frontend): DefaultFrontend(
    (stft): Stft(n_fft=512, win_length=512, hop_length=60, center=True, normalized=False, onesided=True)
    (frontend): Frontend()
    (logmel): LogMel(sr=16000, n_fft=512, n_mels=80, fmin=0, fmax=8000.0, htk=False)
  )
  (normalize): GlobalMVN(stats_file=/net/vol/vivekkan/experiments/fearless/asr1/exp/asr_stats_raw_char/train/feats_stats.npz, norm_means=True, norm_vars=True)
  (encoder): TransformerEncoder(
    (embed): Conv2dSubsampling(
      (conv): Sequential(
        (0): Conv2d(1, 256, kernel_size=(3, 3), stride=(2, 2))
        (1): ReLU()
        (2): Conv2d(256, 256, kernel_size=(3, 3), stride=(2, 2))
        (3): ReLU()
      )
      (out): Sequential(
        (0): Linear(in_features=15360, out_features=256, bias=True)
        (1): PositionalEncoding(
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (encoders): MultiSequential(
      (0): EncoderLayer(
        (self_attn): MultiHeadedAttention(
    